<a href="https://colab.research.google.com/github/adrian-ai-salud/adrian-ai-salud/blob/main/Estudio%20UNIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis del estudio UNIR IA

# Análisis de fiabilidad del cuestionario

In [20]:
# Paso 1: Subir el archivo CSV manualmente
from google.colab import files
uploaded = files.upload()

import pandas as pd
import numpy as np
import pingouin as pg

# Leer archivo (el primero subido)
filename = next(iter(uploaded))
df = pd.read_csv(filename)

# Paso 2: Seleccionar solo las columnas tipo Likert (preguntas 1 a 10)
cols_likert = [col for col in df.columns if col.strip().startswith(tuple([f"{i}." for i in range(1, 11)]))]
df_likert = df[cols_likert].copy()

# Paso 3: Limpieza de respuestas (espacios, mayúsculas, errores ortográficos)
def limpiar(texto):
    texto = str(texto).strip().lower()
    if "moi en desacordo" in texto or "totalemnte en desacordo" in texto or "totalmente en desacordo" in texto:
        return "moi en desacordo"
    elif "en desacordo" in texto:
        return "en desacordo"
    elif "nin de acordo" in texto:
        return "nin de acordo nin en desacordo"
    elif "de acordo" in texto and "totalmente" not in texto:
        return "de acordo"
    elif "totalmente de acordo" in texto:
        return "totalmente de acordo"
    else:
        return np.nan

df_likert = df_likert.applymap(limpiar)

# Paso 4: Convertir a escala numérica (1-5)
likert_map = {
    "moi en desacordo": 1,
    "en desacordo": 2,
    "nin de acordo nin en desacordo": 3,
    "de acordo": 4,
    "totalmente de acordo": 5
}
df_likert = df_likert.replace(likert_map)

# Paso 5: Separar constructos
constructo1_cols = [cols_likert[i] for i in [0,1,2,3,4,6]]  # ítems 1,2,3,4,5,7
constructo2_cols = [cols_likert[i] for i in [5,7,8,9]]      # ítems 6,8,9,10

constructo1 = df_likert[constructo1_cols].dropna()
constructo2 = df_likert[constructo2_cols].dropna()

# Paso 6: Calcular alfa de Cronbach
alpha1 = pg.cronbach_alpha(constructo1)[0]
alpha2 = pg.cronbach_alpha(constructo2)[0]

# Resultados
print(f"✅ Alfa de Cronbach Constructo 1: {round(alpha1, 3)} — n = {constructo1.shape[0]}")
print(f"✅ Alfa de Cronbach Constructo 2: {round(alpha2, 3)} — n = {constructo2.shape[0]}")


Saving Estudio UNIR.csv to Estudio UNIR (2).csv
✅ Alfa de Cronbach Constructo 1: 0.79 — n = 102
✅ Alfa de Cronbach Constructo 2: 0.328 — n = 102


<ipython-input-20-1aee6e58d0cc>:33: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_likert = df_likert.applymap(limpiar)
<ipython-input-20-1aee6e58d0cc>:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_likert = df_likert.replace(likert_map)


In [17]:
import scipy.stats as stats

def resumen_descriptivo_con_ic(data, alpha=0.05):
    resumen = pd.DataFrame(columns=["Media", "Desviación típica", "IC 95% (Límite inferior)", "IC 95% (Límite superior)"])

    for col in data.columns:
        x = data[col].dropna()
        media = x.mean()
        std = x.std(ddof=1)
        n = len(x)
        se = std / np.sqrt(n)
        t_crit = stats.t.ppf(1 - alpha/2, df=n-1)
        ci_lower = media - t_crit * se
        ci_upper = media + t_crit * se

        resumen.loc[col.strip()] = [round(media, 2), round(std, 2), round(ci_lower, 2), round(ci_upper, 2)]

    return resumen

# Calcular para ambos constructos
resumen_c1 = resumen_descriptivo_con_ic(constructo1)
resumen_c2 = resumen_descriptivo_con_ic(constructo2)

# Mostrar resultados
print("📊 Resumen descriptivo - Constructo 1")
display(resumen_c1)
print(f"ℹ️ N total de casos válidos: {constructo1.shape[0]}")

print("\n📊 Resumen descriptivo - Constructo 2")
display(resumen_c2)
print(f"ℹ️ N total de casos válidos: {constructo2.shape[0]}")


📊 Resumen descriptivo - Constructo 1


,Media,Desviación típica,IC 95% (Límite inferior),IC 95% (Límite superior)
1.Creo que o uso da IA na miña especialidade podería mellorar a atención aos pacientes.,3.90,1.01,3.70,4.10
2. Creo que o uso da IA na miña especialidade podería mellorar a toma de decisións clínicas,3.92,0.96,3.73,4.11
3.Creo que a IA pode mellorar os indicadores de saúde da poboación,3.84,1.06,3.63,4.05
4.Creo que a IA pode cambiar o meu papel como profesional sanitario no futuro,3.73,1.14,3.50,3.95
5.Creo que a introdución da IA pode reducir o custo económico asociado ao meu traballo,3.28,1.22,3.04,3.52
7.Creo que a maioría dos profesionais sanitarios están preparados para a introdución da tecnoloxía baseada en IA,2.29,0.99,2.10,2.49


ℹ️ N total de casos válidos: 102

📊 Resumen descriptivo - Constructo 2


,Media,Desviación típica,IC 95% (Límite inferior),IC 95% (Límite superior)
6.Creo que algún día a IA poderá formar parte do meu traballo como profesional sanitario,4.16,0.85,3.99,4.32
8.Creo que recibín unha formación adecuada para usar IA no meu traballo,1.69,0.96,1.50,1.88
9.Creo que existe un marco ético para o uso da tecnoloxía baseada en IA no meu lugar de traballo,2.47,1.22,2.23,2.71
"10.Creo que, se a tecnoloxía baseada en IA comete un erro, toda a responsabilidade recae no profesional sanitario",3.22,1.29,2.96,3.47


ℹ️ N total de casos válidos: 102


In [18]:
from pingouin import cronbach_alpha

def evaluar_alfa_tras_eliminar(data, nombre_constructo):
    print(f"\n🔍 Evaluación del alfa de Cronbach eliminando cada ítem en {nombre_constructo}:\n")
    for col in data.columns:
        subset = data.drop(columns=[col])
        alpha = cronbach_alpha(subset)[0]
        print(f"🗑️ Sin '{col.strip()}': alfa = {round(alpha, 3)}")

# Análisis por constructo
evaluar_alfa_tras_eliminar(constructo1, "Constructo 1")
evaluar_alfa_tras_eliminar(constructo2, "Constructo 2")



🔍 Evaluación del alfa de Cronbach eliminando cada ítem en Constructo 1:

🗑️ Sin '1.Creo que o uso da IA na miña especialidade podería mellorar a atención aos pacientes.': alfa = 0.758
🗑️ Sin '2. Creo que o uso da IA na miña especialidade podería mellorar a toma de decisións clínicas': alfa = 0.734
🗑️ Sin '3.Creo que a IA pode mellorar os indicadores de saúde da poboación': alfa = 0.708
🗑️ Sin '4.Creo que a IA pode cambiar o meu papel como profesional sanitario no futuro': alfa = 0.754
🗑️ Sin '5.Creo que a introdución da IA pode reducir o custo económico asociado ao meu traballo': alfa = 0.765
🗑️ Sin '7.Creo que a maioría dos profesionais sanitarios están preparados para a introdución da tecnoloxía baseada en IA': alfa = 0.817

🔍 Evaluación del alfa de Cronbach eliminando cada ítem en Constructo 2:

🗑️ Sin '6.Creo que algún día a IA poderá formar parte do meu traballo como profesional sanitario': alfa = 0.165
🗑️ Sin '8.Creo que recibín unha formación adecuada para usar IA no meu trabal

In [21]:
# Comprobamos los valores únicos después de limpiar
for col in df_likert.columns:
    print(f"{col}: {df_likert[col].unique()}")


1.Creo que o uso da IA na miña especialidade podería mellorar a atención aos pacientes.  : [5 4 2]
2. Creo que o uso da IA na miña especialidade podería mellorar a toma de decisións clínicas   : [5 4 2]
3.Creo que a IA pode mellorar os indicadores de saúde da poboación  : [5 4 2]
4.Creo que a IA pode cambiar o meu papel como profesional sanitario no futuro  : [5 4 2]
5.Creo que a introdución da IA pode reducir o custo económico asociado ao meu traballo   : [5 2 4 1]
6.Creo que algún día a IA poderá formar parte do meu traballo como profesional sanitario    : [5 4 2]
7.Creo que a maioría dos profesionais sanitarios están preparados para a introdución da tecnoloxía baseada en IA      : [4 2 1 5]
8.Creo que recibín unha formación adecuada para usar IA no meu traballo        : [1 2 5 4]
9.Creo que existe un marco ético para o uso da tecnoloxía baseada en IA no meu lugar de traballo        : [4 2 1 5]
10.Creo que, se a tecnoloxía baseada en IA comete un erro, toda a responsabilidade recae n

In [22]:
df_likert.count()


,0
1.Creo que o uso da IA na miña especialidade podería mellorar a atención aos pacientes.,102
2. Creo que o uso da IA na miña especialidade podería mellorar a toma de decisións clínicas,102
3.Creo que a IA pode mellorar os indicadores de saúde da poboación,102
4.Creo que a IA pode cambiar o meu papel como profesional sanitario no futuro,102
5.Creo que a introdución da IA pode reducir o custo económico asociado ao meu traballo,102
6.Creo que algún día a IA poderá formar parte do meu traballo como profesional sanitario,102
7.Creo que a maioría dos profesionais sanitarios están preparados para a introdución da tecnoloxía baseada en IA,102
8.Creo que recibín unha formación adecuada para usar IA no meu traballo,102
9.Creo que existe un marco ético para o uso da tecnoloxía baseada en IA no meu lugar de traballo,102
"10.Creo que, se a tecnoloxía baseada en IA comete un erro, toda a responsabilidade recae no profesional sanitario",102


In [23]:
print("Constructo 1:", list(constructo1_cols))
print("Constructo 2:", list(constructo2_cols))

print("\nFilas válidas constructo 1:", len(constructo1))
print("Filas válidas constructo 2:", len(constructo2))


Constructo 1: ['1.Creo que o uso da IA na miña especialidade podería mellorar a atención aos pacientes.  ', '2. Creo que o uso da IA na miña especialidade podería mellorar a toma de decisións clínicas   ', '3.Creo que a IA pode mellorar os indicadores de saúde da poboación  ', '4.Creo que a IA pode cambiar o meu papel como profesional sanitario no futuro  ', '5.Creo que a introdución da IA pode reducir o custo económico asociado ao meu traballo   ', '7.Creo que a maioría dos profesionais sanitarios están preparados para a introdución da tecnoloxía baseada en IA      ']
Constructo 2: ['6.Creo que algún día a IA poderá formar parte do meu traballo como profesional sanitario    ', '8.Creo que recibín unha formación adecuada para usar IA no meu traballo        ', '9.Creo que existe un marco ético para o uso da tecnoloxía baseada en IA no meu lugar de traballo        ', '10.Creo que, se a tecnoloxía baseada en IA comete un erro, toda a responsabilidade recae no profesional sanitario  ']

Fi

In [24]:
for col in df_likert.columns:
    print(f"\n📋 {col.strip()}")
    print(df_likert[col].value_counts(dropna=False).sort_index())



📋 1.Creo que o uso da IA na miña especialidade podería mellorar a atención aos pacientes.
1.Creo que o uso da IA na miña especialidade podería mellorar a atención aos pacientes.  
2    19
4    55
5    28
Name: count, dtype: int64

📋 2. Creo que o uso da IA na miña especialidade podería mellorar a toma de decisións clínicas
2. Creo que o uso da IA na miña especialidade podería mellorar a toma de decisións clínicas   
2    17
4    59
5    26
Name: count, dtype: int64

📋 3.Creo que a IA pode mellorar os indicadores de saúde da poboación
3.Creo que a IA pode mellorar os indicadores de saúde da poboación  
2    22
4    52
5    28
Name: count, dtype: int64

📋 4.Creo que a IA pode cambiar o meu papel como profesional sanitario no futuro
4.Creo que a IA pode cambiar o meu papel como profesional sanitario no futuro  
2    28
4    46
5    28
Name: count, dtype: int64

📋 5.Creo que a introdución da IA pode reducir o custo económico asociado ao meu traballo
5.Creo que a introdución da IA pode red

# Análisis descriptivo de las variables